# 2. Базовые методы обработка текста

Машинное представление текста на естественном языке;  
Предобработка текста: токенизация и сегментация;  
Нормализация слов: стеммеры, лемматизаторы;  
One Hot Encoding и обратный индекс Тf-idf.

### Машинное представление текста на естественном языке 

Представление на уровне символов.  

Если создать строковую переменную, то в памяти компьютера она будет представлена как набор ячеек памяти, содержащих  единицы и нули.

In [34]:
example = 'abcd1234!@#$'
for s in example:
    index = ord(s)
    byte = bin(s.encode()[0])
    print(s, index, byte, sep='\t')

a	97	0b1100001
b	98	0b1100010
c	99	0b1100011
d	100	0b1100100
1	49	0b110001
2	50	0b110010
3	51	0b110011
4	52	0b110100
!	33	0b100001
@	64	0b1000000
#	35	0b100011
$	36	0b100100


Чтобы связать символ с его двоичным представлением, используются кодировки. Про ASCII мы уже упоминали в первой главе. Для кодировки русского текста на ОС Windows часто используют кодировку windows 1251. На ос семейства Linux - utf8. Для кодировки латинских символов требуется один байт на символ, для кириллицы - два байта на символ. 

Если при работе с текстом Вам встретятся крякозябры, то знайте - проблема в неправильной кодировке символов. Компьютер при попытке преобразовать единицы и нули в символ использовал не ту таблицу, поэтому получается набор нечитаемых символов. Кодировку можно сменить в продвинутых текстовых редакторах или средствами python (функции encode и decode).

In [35]:
print('РџСЂРёРІРµС‚'.encode('cp1251').decode('utf-8'))

Привет


Представление на уровне слов.  

Пусть имеется некоторый достаточно большой корпус текста. Мы можем выписать все уникальные слова из этого корпуса, получив подмножество языка, состоящее из нескольких тысяч слов. Некоторые из этих слов не найти в толковом словаре, но в нашем корпусе они встречаются, и что самое важное, они несут в себе определенный смысл. Воспользуемся заранее подготовленным словарем. 

In [50]:
# Read file and strip \n symbols
with open('vocab.txt') as f:
    vocab = f.readlines()
vocab = [s.strip() for s in vocab]

# First 10 words
print(vocab[:10], '...', vocab[-10:], f' Всего в словаре {len(vocab)} слов.')

['a', 'a1', 'a2', 'aa', 'aaa', 'aachen', 'aarhus', 'aaron', 'ab', 'aba'] ... ['zones', 'zoning', 'zoo', 'zoological', 'zoology', 'zoom', 'zu', 'zulu', 'zur', 'zurich']  Всего в словаре 21767 слов.


Простейший метод кодирования слова в языке - это его индекс в словаре.

In [37]:
vocab.index('word')

21516

Предложение тоже можно представить как список индексов, для этого его предварительно нужно сегментировать.  

**Сегментация текста** (text segmentation) - это процесс разделения текста на значимые единицы, такие как слова, фразы и предложения.  
**Токенизация** (tokenize) - частный случай сегментации, в котором разделение основано на четком критерии (обычно по определенному символу).  

Например, разделение текста на предложения можно осуществить, используя точку в качестве разделителя. 

In [43]:
text = 'In the town where I was born. Lived a man who sailed to sea'
tokens = text.split('.')
print(tokens)

['In the town where I was born', ' Lived a man who sailed to sea']


Однако такой алгоритм сломается, если в тексте присутствуют сокращения. На помощь приходят специальные библиотеки, которые реализуют сложные правила и обрабатывают исключения. Мы будем использовать библиотеку nltk

Разделение предложения на слова тоже не тривиальная задача, т.к. не во всех естественных языках присутствуют маркеры границ слов, как пробелы в русском и английском. Например, сегментация предложения "Синхронизация разработки и строительства в Пудуне, Шанхай" на китайском языке выглядит вот так:  

'上海浦东开发与建设同步' → ['上海', '浦东', '开发', ‘与', ’建设', '同步']  

К счастью, в этом курсе мы не будем иметь дело с китайским языком, поэтому можем воспользоваться простейшим способом сегментации предложения - токенизация по пробелу.

In [36]:
sentense = 'In the town where I was born'.lower()
tokens = sentense.split()
print(tokens)

['in', 'the', 'town', 'where', 'i', 'was', 'born']


In [46]:
sent_indexes = [vocab.index(t) for t in tokens]
print(sent_indexes)

[9677, 19574, 19938, 21294, 9493, 21119, 2308]


Что делать со словами, которых нет в словаре?  
Простейший способ решения этой проблемы - использование в словаре специального слова "< UNK >" (unknown, неизвестный). Тогда если в предложении встретится слово, которого нет в словаре, ему будет присвоен индекс слова < UNK >, например 0.  


Другой вопрос: слова машина и машины это один и тот же элемент словаря или разные? Насколько большим будет словарь, если для каждой словоформы использовать отдельный код?

Для решения этой проблемы используется **нормализация** - замена одного слова на другое (нормальное), которое имеет представление в словаре. В общем случае под нормализацией подразумевается две техники: лемматизация и стеммизация. 

**Стемминг** - это простой метод нормализации, чаще всего реализуемый в виде ряда правил, которые постепенно применяются к слову для получения нормализованной формы. Стем - это грубо говоря корень, основа слова. 
Эти правила варьируются от языка к языку и отражают морфологическую структуру используемого языка. Например, для английского возможным правилом может быть удаление буквы “s” в конце слова во множественном числе, чтобы преобразовать его в единственную форму.  
Стемминг в основном используется для индексации документов в поисковой системе, поэтому результатом стемминга могут быть недопустимые слова, например *engine -> engin*. Однако если слово в такой нормальной форме пресутствует в словаре, т.к. обработка идет только внутри для поиска документов и никогда не отображаются пользователю.

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [54]:
stemmer.stem('better')

'better'

**Лемматизация** — процесс приведения словоформы к лемме — её нормальной (словарной) форме. Грубо говоря, это более сложная версия стемминга. Лемматизация сводит каждое слово к его надлежащей базовой форме, то есть к слову, которое мы можем найти в словаре.  
Лемматизация работает точнее, но медленнее, чем стемминг. Продвинутые алгоритмы лемматизации используют контестную информацию для определения части речи слова и правило его приведения к нормальной форме.  

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [50]:
lemmatizer.lemmatize('better', pos='a')

'good'

В русском языке нормальными формами считаются следующие морфологические формы: для существительных — именительный падеж, единственное число; для прилагательных — именительный падеж, единственное число, мужской род; для глаголов, причастий, деепричастий — глагол в инфинитиве (неопределённой форме) несовершенного вида. В отличии от английского, для которого уже придумано множество хороших лемматизаторов, улучшение алгоритмов для русского и других, менее распространенных языков, является активной областью ислледований. 

Прямое использование индекса для числового представления слов не самый лучший выбор, потому что числа лучше подходят для описания количества чего-либо. Более правильный и довольно примитивный способ представить слово в виде вектора длиной, равной длине словаря. Все элементы этого вектора равны нулю, за исключением позиции, которой соответствует индекс этого слова. (Рисунок OHE). 

Представление на уровне предложений  

Как правило, на практике мы имеем дело с предложениями разной длины, что затрудняет их обработку. Мы можем использовать технику One Hot Encoding для кодирования нескольких слов одного предложения или документа. На выходе будет вектор, длина которого равна количеству слов в словаре, а на позициях слов будет количество их вхождений. Т.е. вектор для предложения вычисляется как сумма OHE векторов каждого слова.  

Возникает проблема с наиболее часто встречающимися словами (артиклями, предлогами и т.д.), которые не несут смысловой нагрузки и зашумляют OHE вектор. Для ее решения часто используют две техники:
- выбрасывают стоп-слова
- TF-IDF

TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова $t_{i}$ в пределах отдельного документа. 

$$\mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}$$

где $n_t$ есть число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе.  

IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Учёт IDF уменьшает вес широкоупотребительных слов. Для каждого уникального слова в пределах конкретной коллекции документов существует только одно значение IDF. 

$$\mathrm {idf} (t,D)=\log {\frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}}$$
где

$|D|$ — число документов в коллекции;  
$ |\{\,d_{i}\in D\mid t\in d_{i}\,\}|$ — число документов из коллекции $D$, в которых встречается $t$ (когда $n_{t}\neq 0$).  

Таким образом, мера TF-IDF является произведением двух сомножителей:

$$ \operatorname {tf-idf}(t,d,D)=\operatorname {tf}(t,d)\times \operatorname {idf}(t,D)$$

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.  
Например, если документ содержит 100 слов, и слово «заяц» встречается в нём 3 раза, то частота слова (TF) для слова «заяц» в документе будет 0,03 (3/100).  
Вычислим IDF как десятичный логарифм отношения количества всех документов к количеству документов, содержащих слово «заяц». Таким образом, если «заяц» содержится в 1000 документах из 10 000 000 документов, то IDF будет равной: log(10 000 000/1000) = 4. Для расчета окончательного значения веса слова необходимо TF умножить на IDF. В данном примере, TF-IDF вес для слова «заяц» в выбранном документе будет равен: 0,03 × 4 = 0,12.  


Предобработка текста: токенизация и нормализация. 
Библиотека Nltk.
Векторное представление текста. Библиотека Sklearn.

Ссылки: 
- https://nlpub.ru
- https://habr.com/ru/company/Voximplant/blog/446738/
- Hobson Lane etc, NLP in action, Глава 2. 